In [ ]:
from PosSelect_Functions_Old import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import copy
import seaborn as sns
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
from scipy.stats import fisher_exact
from scipy.stats import norm
from collections import Counter

hfont = {'fontname':'Arial'}
plt.rcParams["font.family"] = "Arial"

#Code borrowed heavily from here: https://stackoverflow.com/questions/62375034/find-non-overlapping-area-between-two-kde-plots
plt.rcParams.update(
    {"text.usetex": False}
)

def parse_table(i):
    return [int(x) for x in i.replace("[", "").replace("]", "").split(",")]

d_abrev = {"LiangSteinNeuron":"FC exc. neur.", "FetalChondrocytes":"F chond.", "SertoliMale":"FG sertoli", "preGC_IIaFemale":"FG preGC IIa",\
          "NeuralFemale":"FG neur.", "FetalGonadImmuneFemale":"FG immune", "VIP":"AC VIP inh. neur.", "LiangSteinProgenitor":"FC prog.",\
          "AdultHeartVentricularCardiomyocyte":"AH cardiomyo.", "AdultLoopOfHenle":"AK loop of henle", "FetalBrainNeurGlioblast_CB_VZ":"FCB glioblast",\
         "AdultProximalTubule":"AK prox. tub.", "FetalLeydigMale":"FG leydig", "SST":"AC SST inh neur.", "KosoyRoussosControlMicroglia":"AC microglia",\
         "FetalBrainFloorPlate":"FB fl. plate", "FetalArterialECs":"FH endoth.", "ASCT":"AC astro.", "FetalBrainCOP":"FB COP",\
         "AMY":"AA neur.", "PVALB":"AC PVALB inh neur.", "ITL23":"AC L2-3 IT neur.", "FetalBrainNeurCB_GNP_IPC_1":"FB inter. prog.", "FetalBrainNeurDAergic":"FB DA neur.",\
          "OGC":"AC Oligo.", "D1Pu":"AP D1 inh neur.", "FetalBrainNeurSerotonergic":"FB 5-HT neur.", "FetalBrainNeurDRG_2":"FS DRG neur.",\
          "FetalHeartPericytes":"FH peri.", "FetalHeartEndocardium":"FH endocard.", "FetalHeartCardiacFibroblasts":"FH fibro.", "FetalBrainNeurPurkinje_6":"FCB Purk. inh neur.",\
          "AdultHeartSmoothMuscle":"AH smooth musc.", "FetalBrainRoofPlate":"FB ro. plate"}

In [ ]:
#Plotting across chromosomes using PhyloP
df = pd.read_csv("PerChrom_Results/PerChrom_PhyloP447_SpecSup250_MAFCut0.25_AllChroms_Bootstrap.txt.gz", sep = "\t")
df = df[df["Proportion"] == 0.6]

medians = []
ci = []
palette = {}
for ct in np.unique(df["Chrom"]):
    dff = df[(df["Chrom"].isin([ct]))]
    dff = dff.sort_values("alpha")
    ci.append([ct, list(dff["alpha"])[25], list(dff["alpha"])[975]])
    medians.append([ct, np.median(dff["alpha"])])
    
df_med = pd.DataFrame(medians).sort_values(1, ascending = False)
order = list(df_med[0])

df_ci = pd.DataFrame(ci)
df_ci.columns = ["Chromosome", "Lower95", "Upper95"]
df_ci = df_ci.sort_values("Lower95", ascending = False)

df.columns = ["Chromosome"] + list(df.columns)[1:]

for index, row in df_ci.iterrows():
    if row["Lower95"] > 0:
        palette[row["Chromosome"]] = "#F42FF5"
    else:
        palette[row["Chromosome"]] = "#40A94D"

fig, ax = plt.subplots(figsize = (10, 4))
sns.boxplot(data = df, y = "alpha", x = "Chromosome", order = order, palette = palette)
plt.ylim(-0.15, 0.075)
plt.ylabel("$\\alpha_{Cons}$", size = 16)
plt.xlabel("Chromosome", size = 16)
plt.title("Positive selection on non-coding sites per chromosome", size = 20)
plt.xticks(rotation = 90, size = 18)
#plt.title("Positive selection in HARs across cell types", size = 25)

df_ci

In [ ]:
#Exploring the chromosome X stuff
v, yvalls = read_noncoding_data_fast(path = "./", maf_cut = 0.25, spec_sup = 250)

yvalls.index = yvalls["Position"]
yvalls = add_unfold(yvalls)

In [ ]:
#Because vv is faster to type
vv = yvalls.copy()
yvalls = 0

In [ ]:
#Restrict to chrX
v["Chrom"] = [x.split(":")[0] for x in v["Position"]]
vv["Chrom"] = [x.split(":")[0] for x in vv["Position"]]

vx = v[v["Chrom"].isin(["chrX"])]
vvx = vv[vv["Chrom"].isin(["chrX"])]


In [ ]:
#Plot chrX
vvv = prepare_alpha(vx, vvx)
z = [np.float64(j) for j in list(vvx["PhyloP447"])]
z.sort()
cuttt = 0.95
cutoff = z[int(floor((len(z)*cuttt)))]

alpha = compute_alpha_cutoff(vvv, dn_cut = 0.01, plot = True, cutoff = cutoff, title = "Excess of fixed substitutions in conserved sites on chrX")
table2 = alpha[3]
table1 = alpha[2]

p = (fisher_exact(table1, alternative = "greater")[1] + fisher_exact(table2, alternative = "greater")[1])/2
print(alpha, p)

In [ ]:

vvv = prepare_alpha(vx, vvx)
z = [np.float64(j) for j in list(vvx["PhyloP447"])]
z.sort()
cuttt = 0.6
cutoff = z[int(floor((len(z)*cuttt)))]

alpha = compute_alpha_cutoff(vvv, dn_cut = 0.01, plot = True, cutoff = cutoff, title = "Excess of fixed substitutions in conserved sites on chrX", window = [3, 12])
table2 = alpha[3]
table1 = alpha[2]

p = (fisher_exact(table1, alternative = "greater")[1] + fisher_exact(table2, alternative = "greater")[1])/2
plt.title("")
plt.xlabel("")
plt.ylabel("")
plt.xticks([], [])
plt.yticks([], [])
plt.legend([], [], frameon = False)

In [ ]:
#Read in the chrX selective sweep regions
kl = pd.read_csv("KasperLange_CellReports2023_chrX.csv", header = None)
vx_kl = pd.DataFrame()
vvx_kl = pd.DataFrame()

for index, row in kl.iterrows():
    vx_kl = pd.concat([vx_kl, vx[vx["Position"].isin(["chrX:" + str(i) for i in range(row[1], row[2])])]])
    vvx_kl = pd.concat([vvx_kl, vvx[vvx["Position"].isin(["chrX:" + str(i) for i in range(row[1], row[2])])]])

In [ ]:
#Test on those regions
vvv = prepare_alpha(vx_kl, vvx_kl, stat = "PhyloP447")
cuttt = 0.95
z = list(vvx_kl["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Known selective sweeps")
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

In [ ]:
#Test on those regions
vvv = prepare_alpha(vx_kl, vvx_kl, stat = "PhyloP447")
cuttt = 0.6
z = list(vvx_kl["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Known selective sweeps", window = [3, 12])
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

plt.title("")
plt.xlabel("")
plt.ylabel("")
plt.xticks([], [])
plt.yticks([], [])
plt.legend([], [], frameon = False)

In [ ]:
#Test on everything without those regions
vx_nkl = vx[~vx["Position"].isin(vx_kl["Position"])].copy()
vvx_nkl = vvx[~vvx["Position"].isin(vvx_kl["Position"])].copy()
vvv = prepare_alpha(vx_nkl, vvx_nkl, stat = "PhyloP447")
cuttt = 0.95
z = list(vvx_nkl["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Not in known selective sweeps")
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

In [ ]:
#Test on everything without those regions
vx_nkl = vx[~vx["Position"].isin(vx_kl["Position"])].copy()
vvx_nkl = vvx[~vvx["Position"].isin(vvx_kl["Position"])].copy()
vvv = prepare_alpha(vx_nkl, vvx_nkl, stat = "PhyloP447")
cuttt = 0.9
z = list(vvx_nkl["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Not in known selective sweeps", window = [3, 12])
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

plt.title("")
plt.xlabel("")
plt.ylabel("")
plt.xticks([], [])
plt.yticks([], [])
plt.legend([], [], frameon = False)

In [ ]:
#Test only on those regions near our multi-copy and ampliconic expansions
amp = pd.read_csv("chrX_Ampliconic.csv")
amp = amp[amp["Gene class"].isin(["Multicopy", "Ampliconic"])]

vx_amp = vx[vx["NearestGene"].isin(amp["Gene name"])]
vvx_amp = vvx[vvx["NearestGene"].isin(amp["Gene name"])]

#Test on those regions
vvv = prepare_alpha(vx_amp, vvx_amp, stat = "PhyloP447")
cuttt = 0.95
z = list(vvx_amp["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Near known multicopy testis-expressed genes")
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

In [ ]:
#Test on those regions
vvv = prepare_alpha(vx_amp, vvx_amp, stat = "PhyloP447")
cuttt = 0.9
z = list(vvx_amp["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Near known multicopy testis-expressed genes", window = [3, 12])
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

plt.title("")
plt.xlabel("")
plt.ylabel("")
plt.xticks([], [])
plt.yticks([], [])
plt.legend([], [], frameon = False)

In [ ]:
#Test only on anything not in those regions near our multi-copy and ampliconic expansions
vx_namp = vx[~vx["Position"].isin(vx_amp["Position"])].copy()
vvx_namp = vvx[~vvx["Position"].isin(vvx_amp["Position"])].copy()
vvv = prepare_alpha(vx_nkl, vvx_nkl, stat = "PhyloP447")
cuttt = 0.9
z = list(vvx_amp["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Not near known multicopy testis-expressed genes")
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

In [ ]:
#Test only on anything not in those regions near our multi-copy and ampliconic expansions
vx_namp = vx[~vx["Position"].isin(vx_amp["Position"])].copy()
vvx_namp = vvx[~vvx["Position"].isin(vvx_amp["Position"])].copy()
vvv = prepare_alpha(vx_nkl, vvx_nkl, stat = "PhyloP447")
cuttt = 0.9
z = list(vvx_amp["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Not near known multicopy testis-expressed genes", window = [3, 12])
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

plt.title("")
plt.xlabel("")
plt.ylabel("")
plt.xticks([], [])
plt.yticks([], [])
plt.legend([], [], frameon = False)

In [ ]:
vx_use = vx[(vx["NearestGene"].isin(amp["Gene name"])) | (vx["Position"].isin(vx_kl["Position"]))]
vvx_use = vvx[(vvx["NearestGene"].isin(amp["Gene name"])) | (vvx["Position"].isin(vvx_kl["Position"]))]

#Test on those regions
vvv = prepare_alpha(vx_use, vvx_use, stat = "PhyloP447")
cuttt = 0.6
z = list(vvx_amp["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Known hotspots of positive selection on chrX")
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

In [ ]:
vx_use = vx[vx["NearestGene"].isin(["LAS1L"])]
vvx_use = vvx[vvx["NearestGene"].isin(["LAS1L"])]

#Test on those regions
vvv = prepare_alpha(vx_use, vvx_use, stat = "PhyloP447")
cuttt = 0.9
z = list(vvx_amp["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Known hotspots of positive selection on chrX", window = [-5, 12])
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

plt.title("")
plt.xlabel("")
plt.ylabel("")
plt.xticks([], [])
plt.yticks([], [])
plt.legend([], [], frameon = False)

In [ ]:
vx_use = vx[(vx["NearestGene"].isin(amp["Gene name"])) | (vx["Position"].isin(vx_kl["Position"]))]
vvx_use = vvx[(vvx["NearestGene"].isin(amp["Gene name"])) | (vvx["Position"].isin(vvx_kl["Position"]))]

#Test on those regions
vvv = prepare_alpha(vx_use, vvx_use, stat = "PhyloP447")
cuttt = 0.6
z = list(vvx_amp["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Known hotspots of positive selection on chrX", window = [3, 12])
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

plt.title("")
plt.xlabel("")
plt.ylabel("")
plt.xticks([], [])
plt.yticks([], [])
plt.legend([], [], frameon = False)

In [ ]:
vx_use = vx[~((vx["NearestGene"].isin(amp["Gene name"])) | (vx["Position"].isin(vx_kl["Position"])))]
vvx_use = vvx[~((vvx["NearestGene"].isin(amp["Gene name"])) | (vvx["Position"].isin(vvx_kl["Position"])))]

#Test on those regions
vvv = prepare_alpha(vx_use, vvx_use, stat = "PhyloP447")
cuttt = 0.6
z = list(vvx_amp["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Outside known hotspots of positive selection on chrX")
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

In [ ]:
#Test on those regions
vvv = prepare_alpha(vx_use, vvx_use, stat = "PhyloP447")
cuttt = 0.6
z = list(vvx_amp["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Known hotspots of positive selection on chrX", window = [3, 12])
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

plt.title("")
plt.xlabel("")
plt.ylabel("")
plt.xticks([], [])
plt.yticks([], [])
plt.legend([], [], frameon = False)

In [ ]:
vx_use = vx[~((vx["NearestGene"].isin(amp["Gene name"])) | (vx["Position"].isin(vx_kl["Position"])))]
vvx_use = vvx[~((vvx["NearestGene"].isin(amp["Gene name"])) | (vvx["Position"].isin(vvx_kl["Position"])))]

#Test on those regions
vvv = prepare_alpha(vx_use, vvx_use, stat = "PhyloP447")
cuttt = 0.6
z = list(vvx_amp["PhyloP447"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, plot = True, cutoff = cutoff, title = "Outside known hotspots of positive selection on chrX", window = [3, 12])
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)
print(alpha)

plt.title("")
plt.xlabel("")
plt.ylabel("")
plt.xticks([], [])
plt.yticks([], [])
plt.legend([], [], frameon = False)

In [ ]:
hpo = pd.read_csv("../DPSC_CNCC/HPO_AccelEvol_Input.txt", sep= "\t")
d_HPO = {}

for index, row in hpo.iterrows():
    d_HPO[row["Term"]] = row["Genes"].split(";")

gobp = pd.read_csv("../DPSC_CNCC/GOBP_AccelEvol_Input.txt", sep= "\t")
d_BP = {}

for index, row in gobp.iterrows():
    if "ucus" in row["Term"]:
        print(row["Term"])
    d_BP[row["Term"]] = row["Genes"].split(";")
    
kegg = pd.read_csv("../DPSC_CNCC/KEGG_AccelEvol_Input.txt", sep= "\t")
d_KEGG = {}

for index, row in kegg.iterrows():
    if "ucus" in row["Term"]:
        print(row["Term"])
    d_KEGG[row["Term"]] = row["Genes"].split(";")

In [ ]:
#Test per geneset on chrX
c = 0
out = []
vx.index = vx["NearestGene"]
vvx.index = vvx["NearestGene"]

for key in d_HPO.keys():
    if c%1000 == 0:
        print(c)
    if len(d_HPO[key]) >= 15 and len(d_HPO[key]) < 100:
        vk = vx.loc[np.intersect1d(np.intersect1d(vx.index, d_HPO[key]), vvx.index)]
        vvk = vvx.loc[np.intersect1d(np.intersect1d(vvx.index, d_HPO[key]), vx.index)]
        x2 = [np.float64(x) for x in list(vk["PhyloP447"])]
        yvals2 = [np.float64(j) for j in list(vvk["PhyloP447"])]
        yvals2.sort()
        if len(vk.index) >= 100 and len(vvk.index) >= 50:
            vvv = prepare_alpha(vk, vvk, stat = "PhyloP447")
            for cuttt in [0.6, 0.7, 0.8, 0.9]:
                cutoff = yvals2[int(floor((len(yvals2)*cuttt)))]
                alpha = compute_alpha_cutoff(vvv, dn_cut = 0.05, plot = False, cutoff = cutoff)

                table1 = alpha[2]
                table2 = alpha[3]
                
                out.append([key, np.median(x2), len(x2), np.median(yvals2), len(yvals2), (fisher_exact(table1)[1] + fisher_exact(table2)[1])/2, mwu(x2, yvals2)[1], (fisher_exact(table1, alternative = "greater")[1] + fisher_exact(table2, alternative = "greater")[1])/2, mwu(x2, yvals2, alternative = "greater")[1], alpha[0], alpha[1], table1, table2, cuttt])
    c += 1
    
from statsmodels.stats.multitest import fdrcorrection
df = pd.DataFrame(out)
df = df.sort_values(7)
df.columns = ["Term", "Median Fixed PhyloP", "Number Fixed Variants", "Median Polymorphic PhyloP", "Number Polymorphic Variants", "Fisher exact p-value", "MWU p-value", "Fisher exact p-value; alt greater", "MWU p-value; alt greater", "alpha", "Cutoff", "[[dc1, du1], [pc1, pu1]]", "[[dc2, du2], [pc2, pu2]]", "Proportion"]
df.to_csv("Finalizing_FisherExact_Unfold_NewHPO_NonCod_MafCut0.25-0.75_chrX_NonCod_SpecSup250.csv", index = False)

In [ ]:
df = pd.read_csv("ForPub_DoubleIntersect_FisherExact_Unfold_NewHPO_NonCod_MafCut0.25-0.75_chrX_NonCod_SpecSup250.csv")
new_df = pd.DataFrame()
for prop in np.unique(df["Proportion"]):
    df2 = df[df["Proportion"] == prop]
    df2["Fisher exact FDR"] = fdrcorrection(df2["Fisher exact p-value; alt greater"])[1]
    df2["MWU FDR"] = fdrcorrection(df2["MWU p-value; alt greater"])[1]
    new_df = pd.concat([new_df, df2])
df = new_df.sort_values("Fisher exact FDR")
df_sig = df[df["Fisher exact FDR"] < 0.1]
df_sig2 = df[df["MWU FDR"] < 0.1]
df_sig

In [ ]:
key = "Short toe"

vx.index = vx["NearestGene"]
vvx.index = vvx["NearestGene"]

vk = vx.loc[np.intersect1d(np.intersect1d(vx.index, d_HPO[key]), vvx.index)]
vvk = vvx.loc[np.intersect1d(np.intersect1d(vvx.index, d_HPO[key]), vx.index)]
x2 = [np.float64(x) for x in list(vk["PhyloP447"])]
yvals2 = [np.float64(j) for j in list(vvk["PhyloP447"])]
yvals2.sort()
if len(vk.index) >= 100 and len(vvk.index) >= 50:
    vvk = vvk[~vvk["NearestGene"].isin(["SHOX"])]
    vvv = prepare_alpha(vk, vvk, stat = "PhyloP447")
    cuttt = 0.9
    cutoff = yvals2[int(floor((len(yvals2)*cuttt)))]
    alpha = compute_alpha_cutoff(vvv, dn_cut = 0.05, plot = True, cutoff = cutoff, title = key + " genes on chrX")

    table1 = alpha[2]
    table2 = alpha[3]
    print((fisher_exact(table1, alternative = "greater")[1] + fisher_exact(table2, alternative = "greater")[1])/2)
    print((fisher_exact(table1, alternative = "greater")))

In [ ]:

vk = vx.loc[np.intersect1d(vx.index, ["GPC4"])]
vvk = vvx.loc[np.intersect1d(vvx.index, ["GPC4"])]
x2 = [np.float64(x) for x in list(vk["PhyloP447"])]
yvals2 = [np.float64(j) for j in list(vvk["PhyloP447"])]
yvals2.sort()
vvv = prepare_alpha(vk, vvk, stat = "PhyloP447")
cuttt = 0.9
cutoff = yvals2[int(floor((len(yvals2)*cuttt)))]
alpha = compute_alpha_cutoff(vvv, dn_cut = 0.05, plot = True, cutoff = cutoff, title = "$\it{GPC4}$")

table1 = alpha[2]
table2 = alpha[3]
print((fisher_exact(table1, alternative = "greater")[1] + fisher_exact(table2, alternative = "greater")[1])/2)